In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.models.mistral import configuration_mistral as original_configuration_mistral
from transformers.models.mistral import modeling_mistral as original_modeling_mistral

import configuration_mistral
import modeling_mistral

In [3]:


original_modeling_mistral.MistralModel = modeling_mistral.MistralModel
original_modeling_mistral.MistralForCausalLM = modeling_mistral.MistralForCausalLM
original_configuration_mistral.MistralConfig = configuration_mistral.MistralConfig

model_path = "ezelikman/quietstar-8-ahead"

n_ahead = 8
n_ahead_talk = 1
merged_talk_heads = True

print("Loading model")
model = AutoModelForCausalLM.from_pretrained(model_path,
                                             load_in_8bit=True,
                                             max_thoughts=n_ahead + n_ahead_talk + 1,
                                             merged_talk_heads=merged_talk_heads,
                                             merged_lm_and_talk_heads=False,
                                             merged_lm_and_think_heads=True,
                                             use_concat_talk_head=True,
                                             use_shallow_think=True,
                                             use_shallow_talk=False,
                                             use_complex_think_head=False,
                                             use_complex_talk_head=True,
                                             use_weighted_talk_head=True,
                                             )

print("Loading tokenizer")
tokenizer = AutoTokenizer.from_pretrained(model_path)

model.use_end_thought_token = True
model.tokenizer = tokenizer
model.use_start_thought_token = True
model.wandb_enabled = True
model.n_ahead = n_ahead
model.n_passes = 1
model.eval_mode = True
model.first_run = False
model.kill_after = 100
model.rm_initialized = True

model.original_mode = False

input = "Solve the equation 2x + 3x² = 5."

print("Encoding input")
input_ids = tokenizer.encode(input, return_tensors="pt").to(model.device)

# output = model.generate(input_ids, max_length=50)


def generate(input_ids, attention_mask, model, temp, max_length=20):
    with torch.no_grad():
        finished_generating = torch.zeros(len(input_ids), dtype=torch.bool, device=input_ids.device)
        for cur_token_idx in range(max_length):
            print("Current token: {}".format(cur_token_idx))
            # Sample the next token
            new_ids = model(
                input_ids[~finished_generating],
                attention_mask=attention_mask[~finished_generating]
            )['logits']
            # Mask out the start and end thought tokens so we don't accidentally sample them
            new_ids[:, :, model.tokenizer.vocab_size:] = -float("inf")
            for list_idx, answer_idx in enumerate((~finished_generating).nonzero(as_tuple=True)[0]):
                # Find the index of the last token that is not padding
                base_answer_ids = input_ids[answer_idx]
                new_answer_ids = new_ids[list_idx]
                last_token_idx = (base_answer_ids != model.tokenizer.pad_token_id).nonzero(as_tuple=True)[0].max()


                new_ids_sampled = torch.multinomial(
                        torch.nn.functional.softmax(new_answer_ids[last_token_idx] / temp, dim=-1), 1)
                # Assign the new id to the last token
                if last_token_idx + 1 >= len(base_answer_ids):
                    # Add padding everywhere
                    new_padding = torch.full((len(input_ids), 1), model.tokenizer.pad_token_id, dtype=torch.long,
                                             device=input_ids.device)
                    input_ids = torch.cat([input_ids, new_padding], dim=-1)
                    attention_mask = torch.cat([attention_mask, torch.zeros_like(new_padding)], dim=-1)
                attention_mask[answer_idx, last_token_idx + 1] = 1
                input_ids[answer_idx, last_token_idx + 1] = new_ids_sampled
                if new_ids_sampled == model.tokenizer.eos_token_id or new_ids_sampled == model.tokenizer.bos_token_id or new_ids_sampled == model.tokenizer.pad_token_id:
                    finished_generating[answer_idx] = 1
            if finished_generating.all():
                break
    return input_ids, attention_mask


print("Running Generation")
out = generate(input_ids, torch.ones_like(input_ids), model, 0.9, max_length=100)
print(tokenizer.decode(out[0][0], skip_special_tokens=False))

Loading model


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading tokenizer
Encoding input
Running Generation
Current token: 0
Current token: 1
Current token: 2
Current token: 3
Current token: 4
Current token: 5
Current token: 6
Current token: 7
Current token: 8
Current token: 9
Current token: 10
Current token: 11
Current token: 12
Current token: 13
Current token: 14
Current token: 15
Current token: 16
Current token: 17
Current token: 18
Current token: 19
Current token: 20
Current token: 21
Current token: 22
Current token: 23
Current token: 24
Current token: 25
Current token: 26
Current token: 27
Current token: 28
Current token: 29
Current token: 30
Current token: 31
Current token: 32
Current token: 33
Current token: 34
Current token: 35
Current token: 36
Current token: 37
Current token: 38
Current token: 39
Current token: 40
Current token: 41
Current token: 42
Current token: 43
Current token: 44
Current token: 45
Current token: 46
Current token: 47
Current token: 48
Current token: 49
Current token: 50
Current token: 51
Current token: 52
Curr